In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

## Imports

In [ ]:
pip install pytube pydub

In [ ]:
from matplotlib import pyplot as plt 
import tensorflow as tf 
import tensorflow_io as tfio
from pytube import Playlist, YouTube
from pydub import AudioSegment
import numpy as np
import librosa

## Build Audio Download Function

In [ ]:
"""
twofour = Playlist('https://www.youtube.com/watch?v=Q6JDOyDqOFs&list=PLel4r-3d96fa-eKRp7CIW6I6xWFW5-3Dx&pp=iAQB')

twofour.videos[1].streams.filter(only_audio=True) # get itags for audio

for video in twofour.videos[0:100]:
    audio = video.streams.get_by_itag(140) # 140 is higher quality audio itag
    audio.download(output_path='2-4 March', filename_prefix='2-4 March ')
    
strathspey = Playlist('https://www.youtube.com/watch?v=EvX_Tkv-j-c&list=PLel4r-3d96fZ9w3RAhwhel5ud8YWk4Jre&pp=iAQB')

for video in strathspey.videos[0:100]:
    audio = video.streams.get_by_itag(140) # 140 is higher quality audio itag
    audio.download(output_path='Strathspey', filename_prefix='Strathspey ')
    
reel = Playlist('https://www.youtube.com/watch?v=TsM7C4ZYmwY&list=PLel4r-3d96fbxHwtJQuZoOkPWve7J02UJ&pp=iAQB')

for video in reel.videos[0:150]:
    audio = video.streams.get_by_itag(140) # 140 is higher quality audio itag
    audio.download(output_path='Reel', filename_prefix='Reel ')
"""

## Build Data Loading Function

In [ ]:
m = '/kaggle/input/march-strathspey-and-reel-files/2-4 March Trimmed/2-4 March 24th Guards Brigade at Anzio.mp3'
s = '/kaggle/input/march-strathspey-and-reel-files/Strathspey Trimmed/Strathspey 3 Strathspeys in the Hoose (Piping 100 Day 48).mp3'
r = '/kaggle/input/march-strathspey-and-reel-files/Reel Trimmed/Reel A Cup of Tea.mp3'

## Define Audio Processing Functions

In [ ]:
def normalize_audio(audio):
    max_val = np.max(np.abs(audio))
    audio_normalized = audio / max_val
    return audio_normalized

def load_wav_16k_mono_from_mp3(filename):
    # Convert MP3 to WAV
    input_file = filename
    output_file = "result.wav"
    sound = AudioSegment.from_mp3(input_file)
    sound.export(output_file, format="wav")

    # Read the WAV file using TensorFlow
    file_contents = tf.io.read_file(output_file)
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)

    # Resample to 16 kHz
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    wav = wav.numpy()

    # Normalize audio
    wav = normalize_audio(wav)

    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=wav, sr=16000, n_mfcc=13)

    return mfccs

## Plot MFCCs

In [ ]:
mwave = load_wav_16k_mono_from_mp3(m)
swave = load_wav_16k_mono_from_mp3(s)
rwave = load_wav_16k_mono_from_mp3(r)

In [ ]:
plt.figure(figsize=(10, 5))
librosa.display.specshow(mwave, x_axis='time')
plt.colorbar()
plt.title('MFCC')
plt.xlabel('Time')
plt.ylabel('MFCC Coefficients')
plt.show()

## Create Tensorflow Datasets

In [ ]:
MARCH = '/kaggle/input/march-strathspey-and-reel-files/2-4 March Trimmed'
STRATHSPEY = '/kaggle/input/march-strathspey-and-reel-files/Strathspey Trimmed'
REEL = '/kaggle/input/march-strathspey-and-reel-files/Reel Trimmed'

In [ ]:
march = tf.data.Dataset.list_files(MARCH+'/*.mp3')
strath = tf.data.Dataset.list_files(STRATHSPEY+'/*.mp3')
reel = tf.data.Dataset.list_files(REEL+'/*.mp3')

## Process All Files

In [ ]:
march_data = []
strath_data = []
reel_data = []

for file_path in march:
    mfccs = load_wav_16k_mono_from_mp3(file_path.numpy().decode("utf-8"))
    march_data.append(mfccs)

for file_path in strath:
    mfccs = load_wav_16k_mono_from_mp3(file_path.numpy().decode("utf-8"))
    strath_data.append(mfccs)

for file_path in reel:
    mfccs = load_wav_16k_mono_from_mp3(file_path.numpy().decode("utf-8"))
    reel_data.append(mfccs)

## Pad MFCCs

In [ ]:
# Find the maximum length of MFCCs across all data
max_length = max(max(mfcc.shape[1] for mfcc in march_data),
                 max(mfcc.shape[1] for mfcc in strath_data),
                 max(mfcc.shape[1] for mfcc in reel_data))

def pad_mfccs(mfcc, max_length):
    # Pad the MFCCs with zeros to match the max_length
    padded_mfcc = np.pad(mfcc, ((0, 0), (0, max_length - mfcc.shape[1])), 'constant')
    return padded_mfcc

# Pad all MFCCs to the maximum length
march_data = [pad_mfccs(mfcc, max_length) for mfcc in march_data]
strath_data = [pad_mfccs(mfcc, max_length) for mfcc in strath_data]
reel_data = [pad_mfccs(mfcc, max_length) for mfcc in reel_data]

# Verify all MFCCs are correctly padded
for mfcc in march_data + strath_data + reel_data:
    assert mfcc.shape[1] == max_length, "Padding error: MFCC lengths are inconsistent"

## Generate Tensorflow Datasets

In [ ]:
def generator(data_list, indices):
    for index in indices:
        yield data_list[index]

# Create TensorFlow datasets
def create_dataset(data):
    indices = list(range(len(data)))
    dataset = tf.data.Dataset.from_generator(
        generator, 
        args=(data, indices), 
        output_signature=tf.TensorSpec(shape=(13, max_length), dtype=tf.float32)
    )
    return dataset

march_dataset = create_dataset(march_data)
strath_dataset = create_dataset(strath_data)
reel_dataset = create_dataset(reel_data)

marches = tf.data.Dataset.zip((march_dataset, tf.data.Dataset.from_tensor_slices(tf.zeros(len(march), dtype=tf.int32))))  # Label march as 0
strathspeys = tf.data.Dataset.zip((strath_dataset, tf.data.Dataset.from_tensor_slices(tf.ones(len(strath), dtype=tf.int32))))  # Label strathspey as 1
reels = tf.data.Dataset.zip((reel_dataset, tf.data.Dataset.from_tensor_slices(tf.ones(len(reel), dtype=tf.int32) * 2)))  # Label reel as 2

data = marches.concatenate(strathspeys).concatenate(reels)

## Split and Label Datasets

In [ ]:
from sklearn.model_selection import train_test_split

# Convert the concatenated dataset to a list of tuples
all_data_padded = list(data)

# Extract features and labels
features = [item[0].numpy() for item in all_data_padded]
labels = [item[1].numpy() for item in all_data_padded]

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(features, labels, train_size=0.65, stratify=labels, random_state=42)

# Convert to TensorFlow datasets
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))

## Train Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
import numpy as np

# 42=66.7%
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

early_stopping = EarlyStopping(patience=8, restore_best_weights=True, monitor='val_accuracy')

# Assume X_train, X_test, y_train, y_test are already defined
# Convert lists to numpy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

# Add a channel dimension
X_train = X_train[..., np.newaxis]  # Shape: (num_samples, 13, max_length, 1)
X_test = X_test[..., np.newaxis]    # Shape: (num_samples, 13, max_length, 1)

input_shape = (13, max_length, 1)  # Ensure max_length is appropriate for your data

model = Sequential([
    Input(shape=input_shape),
    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.005)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.005)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.4),

    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=l2(0.005)),
    BatchNormalization(),
    Dropout(0.5),

    Dense(3, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping])